## Feature_Engineering_2
This is an assignment for my thinkful data science course - "Feature_Engineering_2"
Raj Prasad
May 2019


[html version](https://daddyprasad5.github.io/Feature_Engineering_2.html) - with all the code hidden away for a quick read

[jupyter notebook version](https://github.com/daddyprasad5/thinkful/blob/master/Feature_Engineering_2.ipynb) - with all the code exposed in an interactive notebook

In [106]:
#imports
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings
import math
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
from scipy.stats.mstats import winsorize

%matplotlib inline

In [107]:
#data load
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

raw_used_df = pd.read_sql_query('select * from useducation',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

In [108]:
#fill the missing values

used_df = raw_used_df.copy()

col_list = ["ENROLL", "TOTAL_REVENUE", "FEDERAL_REVENUE",
             "STATE_REVENUE", "LOCAL_REVENUE", "TOTAL_EXPENDITURE",
             "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
             "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE", "GRADES_PK_G",
             "GRADES_KG_G", "GRADES_4_G", "GRADES_8_G", "GRADES_12_G", "GRADES_1_8_G",
             "GRADES_9_12_G", "GRADES_ALL_G"]

states = used_df["STATE"].unique()

for state in states:
    used_df.loc[used_df["STATE"] == state, col_list] = used_df.loc[used_df["STATE"] == state, col_list].interpolate()

# we drop the null values after interpolation
used_df.dropna(inplace=True)

#affirm that there are no more NaNs
n = len(used_df)
cols = used_df.columns
num_null = [len(used_df[used_df[col].isnull()]) / n for col in used_df]
pd.DataFrame([cols, num_null]).transpose()

,0,1
0,PRIMARY_KEY,0
1,STATE,0
2,YEAR,0
3,ENROLL,0
4,TOTAL_REVENUE,0
5,FEDERAL_REVENUE,0
6,STATE_REVENUE,0
7,LOCAL_REVENUE,0
8,TOTAL_EXPENDITURE,0
9,INSTRUCTION_EXPENDITURE,0


In [109]:
#column list for reference
# col_list = ['ENROLL', 'TOTAL_REVENUE',
#        'FEDERAL_REVENUE', 'STATE_REVENUE', 'LOCAL_REVENUE',
#        'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE',
#        'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE',
#        'CAPITAL_OUTLAY_EXPENDITURE', 'GRADES_PK_G', 'GRADES_KG_G',
#        'GRADES_4_G', 'GRADES_8_G', 'GRADES_12_G', 'GRADES_1_8_G',
#        'GRADES_9_12_G', 'GRADES_ALL_G', 'AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE',
#        'AVG_READING_4_SCORE', 'AVG_READING_8_SCORE'] 



1. Create a new score variable from the weighted averages of all score variables in the datasets. **Notice that the number of students in the fourth grade 

In [120]:
#calc weighted average

fourth_weight = used_df['GRADES_4_G']
eighth_weight = used_df['GRADES_8_G']
total_weights = fourth_weight + eighth_weight
fourth_weighted = (used_df['AVG_MATH_4_SCORE'] * fourth_weight + 
                   used_df['AVG_READING_4_SCORE'] * fourth_weight ) * .5
eighth_weighted = (used_df['AVG_MATH_8_SCORE'] * eighth_weight + 
                   used_df['AVG_READING_8_SCORE'] * eighth_weight ) * .5
used_df["weighted_average"]= (fourth_weighted + eighth_weighted) / total_weights


2. What are the correlations between this newly created score variable and the expenditure types? Which one of the expenditure type is more correlated than others?

As one might hope, the instruction expenditure correlates best with the weighted score average. 

In [126]:
scores_and_exp = used_df[['weighted_average','TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE', 
         'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE', 
         'CAPITAL_OUTLAY_EXPENDITURE']]
scores_and_exp.corr()

,weighted_average,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
weighted_average,1.000000,0.074155,0.087386,0.079002,0.010959,-0.005389
TOTAL_EXPENDITURE,0.074155,1.000000,0.990255,0.992988,0.943702,0.925784
INSTRUCTION_EXPENDITURE,0.087386,0.990255,1.000000,0.974399,0.902173,0.884798
SUPPORT_SERVICES_EXPENDITURE,0.079002,0.992988,0.974399,1.000000,0.952282,0.911245
OTHER_EXPENDITURE,0.010959,0.943702,0.902173,0.952282,1.000000,0.918188
CAPITAL_OUTLAY_EXPENDITURE,-0.005389,0.925784,0.884798,0.911245,0.918188,1.000000


3. Now, apply PCA to the four expenditure types. How much of the total variance is explained by the first component?

94%

In [151]:
#Calc PCA

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Normalize the data so that all variables have a mean of 0 and standard deviation
# of 1.
exp = used_df[['INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 
               'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE']]
X = StandardScaler().fit_transform(exp)

# execute PCA analysis for just the first dimension
sklearn_pca = PCA(n_components=1)
Y_sklearn = sklearn_pca.fit_transform(X)

print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    sklearn_pca.explained_variance_ratio_
)



The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 [0.9430175]


4. What is the correlation between the overall score variable and the first principal component? 

0.045

In [157]:
#add the first PCA component to the df
scores_and_exp["Y_sklearn"] = Y_sklearn
scores_and_exp.corr()

,weighted_average,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,Y_sklearn
weighted_average,1.000000,0.074155,0.087386,0.079002,0.010959,-0.005389,0.044596
TOTAL_EXPENDITURE,0.074155,1.000000,0.990255,0.992988,0.943702,0.925784,0.992054
INSTRUCTION_EXPENDITURE,0.087386,0.990255,1.000000,0.974399,0.902173,0.884798,0.968606
SUPPORT_SERVICES_EXPENDITURE,0.079002,0.992988,0.974399,1.000000,0.952282,0.911245,0.988345
OTHER_EXPENDITURE,0.010959,0.943702,0.902173,0.952282,1.000000,0.918188,0.971343
CAPITAL_OUTLAY_EXPENDITURE,-0.005389,0.925784,0.884798,0.911245,0.918188,1.000000,0.955792
Y_sklearn,0.044596,0.992054,0.968606,0.988345,0.971343,0.955792,1.000000


5. If you were to choose the best variables for your model, would you prefer using the first principal component instead of the expenditure variables? Why?

I would prefer using the instructional and support expenditure variables directly - or even just one of the two - rather than the PCA component, because the PCA component's correlation with the target variable is not as high as those others. 